In [1]:
import json
import os
import time
import pandas as pd
import random
import time
import pickle

import numpy as np
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding


from keras.backend import clear_session
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
K.set_session(session)

Using TensorFlow backend.


In [2]:

with open("bb_train<20.json") as f:
    all_data=json.load(f)
print('num samples: ',len(all_data))


#cant be too large, will trigger a memory error later on. Failed at 1 Mill
#np.random.shuffle(all_data)

data = all_data[:1000000]


"""


#Need to optimize this later, replace use np arrays 
#instead of python lists

top_k = 10000

t = Tokenizer(num_words=top_k, oov_token='<unk>')
t.fit_on_texts(data)
"""

with open('bb-v1-tok.pickle', 'rb') as handle:
    t = pickle.load(handle)
    


start = time.time()
seqs = [t.texts_to_sequences(x) for x in data]
print('elaspted text->int seqs: ',time.time() - start)

seq_data = list()
for seq in seqs:
    flat = [num for sublist in seq for num in sublist]
    seq_data.append(flat)
    
print('total seq construction time: ',time.time() - start)

num samples:  55873682
elaspted text->int seqs:  47.3108069896698
total seq construction time:  49.86976599693298


Need to optimize this later, use np arrays instead of python lists


For reference, seqs took 57s to go through 1 mill samples

In [3]:
max_len = max([len(seq) for seq in seq_data])

sequences = pad_sequences(seq_data, maxlen=max_len, padding='pre')

print('Max Seq Len: %d' % max_len)

sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
vocab_sz = len(t.word_index)+1
print('vocab sz: ',vocab_sz)

#y_cat = to_categorical(y, num_classes=vocab_sz) #1-hot encoding 

Max Seq Len: 19
vocab sz:  40497


In [4]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.20, random_state=7)

#NN definition
model = Sequential()
model.add(Embedding(vocab_sz, 300, input_length=max_len-1))
model.add(LSTM(200))
model.add(Dense(vocab_sz, activation='softmax'))
print(model.summary())


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 18, 300)           12149100  
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               400800    
_________________________________________________________________
dense_1 (Dense)              (None, 40497)             8139897   
Total params: 20,689,797
Trainable params: 20,689,797
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

#save model structure as json

#model_json = model.to_json()
#with open("models/huffpo-model-v1.json", "w") as json_file:
#    json_file.write(model_json)

In [6]:

model_json = model.to_json()
with open("models/bb-model-v2.json", "w") as json_file:
    json_file.write(model_json)

fpath = "weights/bb-v2/best-weights.hdf5"
checkpoint = ModelCheckpoint(fpath, monitor='val_loss', verbose=2, save_best_only=True, mode='min')


BS = 100
tb = TensorBoard(log_dir="tensorboard-logs/{}".format('bb-modelv2'))
callback_lst = [checkpoint, tb]
#steps_per_epoch is num of batches that make up 1 epoch, defaults to size of train set
model.fit(X,y,batch_size=BS, validation_split=.15, epochs=10, callbacks=callback_lst, verbose=1)



Instructions for updating:
Use tf.cast instead.
Train on 850000 samples, validate on 150000 samples
Epoch 1/10
850000/850000 [==============================] - 368s 433us/step - loss: 2.7910 - acc: 0.1798 - val_loss: 2.7702 - val_acc: 0.1826

Epoch 00001: val_loss improved from inf to 2.77021, saving model to weights/bb-v2/best-weights.hdf5
Epoch 2/10
850000/850000 [==============================] - 362s 426us/step - loss: 2.7754 - acc: 0.1784 - val_loss: 2.7879 - val_acc: 0.1204

Epoch 00002: val_loss did not improve from 2.77021
Epoch 3/10
850000/850000 [==============================] - 362s 425us/step - loss: 2.7754 - acc: 0.1792 - val_loss: 2.7791 - val_acc: 0.1826

Epoch 00003: val_loss did not improve from 2.77021
Epoch 4/10
850000/850000 [==============================] - 365s 429us/step - loss: 2.7750 - acc: 0.1791 - val_loss: 2.7785 - val_acc: 0.1826

Epoch 00004: val_loss did not improve from 2.77021
Epoch 5/10
850000/850000 [==============================] - 362s 426us/step